In [14]:
from pyscf import gto,scf
import pyscf
from alch_deriv import alch_deriv
import matplotlib.pyplot as plt
from scipy.interpolate import InterpolatedUnivariateSpline as spl
import numpy as np
from FcMole import FcM
from aaff import aaff
from pyscf.geomopt.geometric_solver import optimize
ang2bohr=1.8897261246
bohr2ang=.5291772109

In [15]:
CO=gto.M(atom="C 0 0 0; O 0 0 2.05",unit='Bohrs',basis="ccpcvtz",verbose=0)
hfCO=scf.RHF(CO)
CO.nao

86

In [16]:
hfCO.scf()

-112.78189755975957

In [17]:
hfCO.make_rdm1().shape

(86, 86)

In [18]:
def optimizer2at(a1,a2,fcs=None):
    d1=np.linspace(1.9,2.4,9)
    es=[]
    for d in d1:
        if fcs==None:
            mol=gto.M(atom="{} 0 0 0; {} 0 0 {} ".format(a1,a2,d),unit='Bohrs',basis="ccpcvtz",verbose=0)
            mf=scf.RHF(mol)
            es.append(mf.scf())
        else:
            mol=FcM(fcs=fcs,atom="{} 0 0 0; {} 0 0 {} ".format(a1,a2,d),unit='Bohrs',basis="ccpcvtz",verbose=0)
            mf=scf.RHF(mol)
            es.append(mf.scf(dm0=mf.init_guess_by_1e()))
    dspl=np.linspace(1.9,2.4,501)
    bspline=spl(d1,es)
    bspline(dspl)
    dminspline=1.9+.001*(bspline(dspl).argmin())
    d2=np.linspace(dminspline-.03,dminspline+.03,7)
    es=[]
    for d in d2:
        if fcs==None:
            mol=gto.M(atom="{} 0 0 0; {} 0 0 {} ".format(a1,a2,d),unit='Bohrs',basis="ccpcvtz",verbose=0)
            mf=scf.RHF(mol)
            es.append(mf.scf())
        else:
            mol=FcM(fcs=fcs,atom="{} 0 0 0; {} 0 0 {} ".format(a1,a2,d),unit='Bohrs',basis="ccpcvtz",verbose=0)
            mf=scf.RHF(mol)
            es.append(mf.scf(dm0=mf.init_guess_by_1e()))
    dspl=np.linspace(dminspline-.03,dminspline+.03,6001)
    bspline=spl(d2,es)
    #plt.plot(dspl,bspline(dspl))
    COmin=dspl[bspline(dspl).argmin()]
    return COmin,bspline(dspl).min()

In [19]:
optimizer2at("C","O")

(2.08512, -112.78287932476888)

In [20]:
optimizer2at("N","N")

(2.0145099999999996, -108.98769819194126)

In [21]:
optimizer2at("B","F")

(2.35696, -124.15682472205435)

# GAUGE THE ERROR DUE TO BASIS SET

In [22]:
#CO@NN
optimizer2at("N","N",fcs=[-1,1.])

(2.0835500000000002, -112.76197996807296)

In [23]:
#CO@BF
optimizer2at("B","F",fcs=[1.,-1.])

(2.0903899999999997, -112.76101585425657)

In [24]:
#BF@CO
optimizer2at("C","O",fcs=[-1,1])

(2.35566, -124.13411787480872)

In [25]:
#NN@CO
optimizer2at("C","O",fcs=[1,-1])

(2.0167699999999997, -108.96794872500617)